In [14]:
import copy
import cv2 as cv
import imageio
from multiprocessing import Pool
import numpy as np
import os
import open3d as o3d
import pickle
import scipy
import shutil
from tqdm import tqdm

In [35]:
# Scaling coefficient for depth, the same as in TUM
DEPTH_SCALE_FACTOR = 5000
SETUP_CONFIG = 'bandeja_standard.pickle'

# Folder where the data in TUM format will be put
final_folder = '2021-03-01-15-37-25'

azure_depth_folder = '../bag-extractor/2021-03-01-15-37-25/_azure_depth_image_raw/'
smartphone_folder = '../bag-extractor/2021-03-01-15-37-25/smartphone_video_frames/'

In [36]:
depth_ts = np.array([int(file.split('.')[0]) for file in os.listdir(azure_depth_folder) 
                     if file.endswith('.npy')])
depth_ts.sort()

rgb_ts = np.array([int(file.split('.')[0]) for file in sorted(os.listdir(smartphone_folder)) 
                   if file.endswith('.png')])
rgb_ts.sort()

print('Depth timestamps from {1} to {2} (cnt {0})'.format(len(depth_ts), depth_ts[0], depth_ts[-1]))
print('RGB timestamps from {1} to {2} (cnt {0})'.format(len(rgb_ts), rgb_ts[0], rgb_ts[-1]))

# Build correspondences between depth and rgb by nearest neighbour algorithm
rgbd_pairs = []
for depth_t in depth_ts:
    closest_rgb_t = min(rgb_ts, key=lambda x: abs(depth_t - x))
    rgbd_pairs.append((closest_rgb_t, depth_t))
    
# Prepare folder infrastructure
if os.path.exists(final_folder):
    shutil.rmtree(final_folder)
os.mkdir(final_folder)
os.mkdir(os.path.join(final_folder, 'depth'))
os.mkdir(os.path.join(final_folder, 'rgb'))

Depth timestamps from 78109803766 to 106309770766 (cnt 142)
RGB timestamps from 74982637670 to 107411736993 (cnt 973)


In [37]:
def process_pair(rgbd_pair):
    rgb_image = cv.imread(os.path.join(smartphone_folder, str(rgbd_pair[0]) + '.png'))
    depth_array = np.load(os.path.join(azure_depth_folder, str(rgbd_pair[1]) + '.npy'))
    
    rgb_image_aligned, depth_array_aligned = align_rgb_depth(rgb_image, depth_array, (0, 0, 1920, 1080))
    
    # Save rgb as 8-bit png
    cv.imwrite(os.path.join(final_folder, 'rgb', str(rgbd_pair[0] / 1e9) + '.png'), rgb_image_aligned)
    
    # Save depth as 16-bit unsigned int with scale factor
    depth_array_aligned = (depth_array_aligned * DEPTH_SCALE_FACTOR).astype(np.uint16)
    imageio.imwrite(os.path.join(final_folder, 'depth', str(rgbd_pair[1] / 1e9) + '.png'), depth_array_aligned)

# Copy pairs of rgb and depth to final dir
with Pool() as pool: 
    pool.map(process_pair, rgbd_pairs)

# Produce file with associations between rgb and depth files
with open(os.path.join(final_folder, 'association.txt'), 'w') as association_file:
    for rgbd_pair in rgbd_pairs:
        association_file.write('{0} {1} {2} {3}\n'
                               .format(rgbd_pair[0] / 1e9, os.path.join('rgb', str(rgbd_pair[0] / 1e9) + '.png'),
                                       rgbd_pair[1] / 1e9, os.path.join('depth', str(rgbd_pair[1] / 1e9) + '.png')))